<h1>
    <center>
        <font color='#F4EEE0'>
            Import Libraries
        </font>
    </center>
</h1>

In [1]:
import pickle
import zipfile
import base64

import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px

from utils import add_remove_characters, get_elevation, generate_base64_id

<h1>
    <center>
        <font color='#F4EEE0'>
            Constants and Variables
        </font>
    </center>
</h1>

In [2]:
RAW_DATA_FILE_PATH = ".\\data\\SynopticData_Iran_2000_2022.zip"
FILE_NAME = "SynopticData_Iran_2000_2022.csv"

<h1>
    <center>
        <font color='#F4EEE0'>
            Load Raw Data
        </font>
    </center>
</h1>

In [3]:
with zipfile.ZipFile(RAW_DATA_FILE_PATH, mode="r") as zf:
    df_raw = pd.read_csv(zf.open(FILE_NAME), sep=";")

In [4]:
df_raw

,name,station_id,lon,lat,ground_elevation,data,tmax,tmin,tm,rrr24_70000
0,Abadan,40831.0,48.214722,30.377222,6.6,2000-01-01 00:00:00,21.4,7.4,13.5,0.0
1,Abadan,40831.0,48.214722,30.377222,6.6,2000-01-02 00:00:00,22.0,7.2,13.3,0.0
2,Abadan,40831.0,48.214722,30.377222,6.6,2000-01-03 00:00:00,23.4,7.0,14.7,0.0
3,Abadan,40831.0,48.214722,30.377222,6.6,2000-01-04 00:00:00,22.0,10.6,16.8,0.0
4,Abadan,40831.0,48.214722,30.377222,6.6,2000-01-05 00:00:00,18.8,11.2,16.8,0.0
...,...,...,...,...,...,...,...,...,...,...
4085815,Zohan,19161.0,59.783300,33.416700,1710.0,2005-12-27 00:00:00,7.2,-3.2,2.0,NaN
4085816,Zohan,19161.0,59.783300,33.416700,1710.0,2005-12-28 00:00:00,5.2,-4.0,0.6,NaN
4085817,Zohan,19161.0,59.783300,33.416700,1710.0,2005-12-29 00:00:00,-1.0,-6.2,-3.6,NaN
4085818,Zohan,19161.0,59.783300,33.416700,1710.0,2005-12-30 00:00:00,-2.0,-7.0,-4.5,NaN


In [5]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4085820 entries, 0 to 4085819
Data columns (total 10 columns):
 #   Column            Dtype  
---  ------            -----  
 0   name              object 
 1   station_id        float64
 2   lon               float64
 3   lat               float64
 4   ground_elevation  float64
 5   data              object 
 6   tmax              float64
 7   tmin              float64
 8   tm                float64
 9   rrr24_70000       float64
dtypes: float64(8), object(2)
memory usage: 311.7+ MB


<h1>
    <center>
        <font color='#F4EEE0'>
            Data Cleansing
        </font>
    </center>
</h1>

<h4>
    <center>
        <font color='#E1F7F5'>
            Rename Columns
        </font>
    </center>
</h4>

In [6]:
print(f"{'Index Column': ^24}{'Name Column': <24}")
for index, name in enumerate(list(df_raw.columns)):
    print(f"{index: ^24}{name: <24}")

      Index Column      Name Column             
           0            name                    
           1            station_id              
           2            lon                     
           3            lat                     
           4            ground_elevation        
           5            data                    
           6            tmax                    
           7            tmin                    
           8            tm                      
           9            rrr24_70000             


In [7]:
df_raw.rename(
    columns={
        "name": "name",
        "station_id": "id",
        "lon": "lon",
        "lat": "lat",
        "ground_elevation": "alt",
        "data": "date",
        "tmax": "tmax",
        "tmin": "tmin",
        "tm": "tm",
        "rrr24_70000": "precip",
    },
    inplace = True
)

<h4>
    <center>
        <font color='#E1F7F5'>
            Convert `date` to Date
        </font>
    </center>
</h4>

In [8]:
df_raw['date'] = pd.to_datetime(df_raw['date'])

<h4>
    <center>
        <font color='#E1F7F5'>
            Clean `name` Column
        </font>
    </center>
</h4>

In [9]:
df_raw['name'] = df_raw['name']\
    .apply(add_remove_characters).str.upper()

<h4>
    <center>
        <font color='#E1F7F5'>
            NA in Information Columns
        </font>
    </center>
</h4>

In [10]:
df_raw.isna().sum()

name            0
id           1915
lon             0
lat             0
alt          2857
date            0
tmax        48871
tmin        82065
tm          19046
precip    1479518
dtype: int64

In [11]:
# `id` column

gb = ["name", "lon", "lat", "alt"]

name_id_na = df_raw[df_raw["id"].isna()]\
    .groupby(gb)\
    .agg(size = ('name', 'size'))\
    .reset_index()

print(name_id_na)

id_default = 10000
i = 1
for _, row in name_id_na.iterrows():
    name = row["name"]
    lon = row["lon"]
    lat = row["lat"]
    id_generate = id_default + i
    df_raw.loc[(df_raw["name"] == name) & (df_raw["lon"] == lon) & (df_raw["lat"] == lat), "id"] = id_generate
    i += 1

       name    lon      lat   alt  size
0  DEHKHODA  48.88  28.1700  18.0   851
1    FARABI  48.60  30.9833   7.0  1064


In [12]:
# `alt` column

gb = ["name", "id", "lon", "lat"]

name_alt_na = df_raw[df_raw["alt"].isna()]\
    .groupby(gb)\
    .agg(size = ('name', 'size'))\
    .reset_index()

print(name_alt_na)

for _, row in name_alt_na.iterrows():
    name = row["name"]
    lon = row["lon"]
    lat = row["lat"]
    alt = get_elevation(lat=lat, lon=lon)
    df_raw.loc[(df_raw["name"] == name) & (df_raw["lon"] == lon) & (df_raw["lat"] == lat), "alt"] = alt

       name       id    lon    lat  size
0  JAFARIEH  18934.0  50.50  34.76  1429
1  JAMKARAN  18956.0  50.91  34.58  1428


In [13]:
df_raw.isna().sum()

name            0
id              0
lon             0
lat             0
alt             0
date            0
tmax        48871
tmin        82065
tm          19046
precip    1479518
dtype: int64

<h1>
    <center>
        <font color='#F4EEE0'>
            Modified Data
        </font>
    </center>
</h1>

In [14]:
data = pd.DataFrame()

gb = ["name", "id", "lat", "lon", "alt"]

stations = df_raw.groupby(gb)\
    .agg(
        num_of_days = ('name', 'size'),
        date_min = ('date', 'min'),
        date_max = ('date', 'max'),
    )\
    .sort_values(by=["name", "id"], ascending=True)\
    .reset_index()

for _, row in stations.iterrows():
    
    st_name = row["name"]
    st_id = row["id"]
    st_lat = row["lat"]
    st_lon = row["lon"]
    st_alt = row["alt"]
    date_min = row["date_min"]
    date_max = row["date_max"]

    df_tmp = df_raw.query(
        expr=f"name == '{st_name}' & id == {st_id} & lat == {st_lat} & lon == {st_lon} & alt == {st_alt}"
    )
    
    df_tmp["date"] = df_tmp["date"].dt.strftime('%Y-%m-%d')
    
    df_date = pd.DataFrame(
        {
            "name": st_name,
            "id": st_id,
            "lat": st_lat,
            "lon": st_lon,
            "alt": st_alt,
            "date": pd.date_range(
                start = date_min.strftime('%Y-%m-%d'),
                end = date_max.strftime('%Y-%m-%d')
            ).strftime('%Y-%m-%d')
        }
    )
    
    df_tmp = df_tmp.merge(right = df_date, on = gb + ["date"])\
        .sort_values(by = ["date"])\
        .reset_index(drop=True)
    
    df_tmp['date'] = pd.to_datetime(df_tmp['date'])

    data = pd.concat([data, df_tmp], axis=0)
    
    del df_tmp, df_date

del df_raw, stations

data.reset_index(drop=True, inplace=True)

C:\Users\psh2610\AppData\Local\Temp\ipykernel_24068\2937961909.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp["date"] = df_tmp["date"].dt.strftime('%Y-%m-%d')
C:\Users\psh2610\AppData\Local\Temp\ipykernel_24068\2937961909.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp["date"] = df_tmp["date"].dt.strftime('%Y-%m-%d')
C:\Users\psh2610\AppData\Local\Temp\ipykernel_24068\2937961909.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [15]:
data

,name,id,lon,lat,alt,date,tmax,tmin,tm,precip
0,ABADAN,40831.0,48.214722,30.377222,6.6,2000-01-01,21.4,7.4,13.5,0.0
1,ABADAN,40831.0,48.214722,30.377222,6.6,2000-01-02,22.0,7.2,13.3,0.0
2,ABADAN,40831.0,48.214722,30.377222,6.6,2000-01-03,23.4,7.0,14.7,0.0
3,ABADAN,40831.0,48.214722,30.377222,6.6,2000-01-04,22.0,10.6,16.8,0.0
4,ABADAN,40831.0,48.214722,30.377222,6.6,2000-01-05,18.8,11.2,16.8,0.0
...,...,...,...,...,...,...,...,...,...,...
4085815,ZOHAN,19161.0,59.783300,33.416700,1710.0,2005-12-27,7.2,-3.2,2.0,NaN
4085816,ZOHAN,19161.0,59.783300,33.416700,1710.0,2005-12-28,5.2,-4.0,0.6,NaN
4085817,ZOHAN,19161.0,59.783300,33.416700,1710.0,2005-12-29,-1.0,-6.2,-3.6,NaN
4085818,ZOHAN,19161.0,59.783300,33.416700,1710.0,2005-12-30,-2.0,-7.0,-4.5,NaN


In [16]:
data["code"] = data.apply(
    lambda x: generate_base64_id(text=str(x["name"]) + str(int(x["id"]))),
    axis=1
)

data

,name,id,lon,lat,alt,date,tmax,tmin,tm,precip,code
0,ABADAN,40831.0,48.214722,30.377222,6.6,2000-01-01,21.4,7.4,13.5,0.0,UVVKQlJFRk9OREE0TXpFPQ==
1,ABADAN,40831.0,48.214722,30.377222,6.6,2000-01-02,22.0,7.2,13.3,0.0,UVVKQlJFRk9OREE0TXpFPQ==
2,ABADAN,40831.0,48.214722,30.377222,6.6,2000-01-03,23.4,7.0,14.7,0.0,UVVKQlJFRk9OREE0TXpFPQ==
3,ABADAN,40831.0,48.214722,30.377222,6.6,2000-01-04,22.0,10.6,16.8,0.0,UVVKQlJFRk9OREE0TXpFPQ==
4,ABADAN,40831.0,48.214722,30.377222,6.6,2000-01-05,18.8,11.2,16.8,0.0,UVVKQlJFRk9OREE0TXpFPQ==
...,...,...,...,...,...,...,...,...,...,...,...
4085815,ZOHAN,19161.0,59.783300,33.416700,1710.0,2005-12-27,7.2,-3.2,2.0,NaN,V2s5SVFVNHhPVEUyTVE9PQ==
4085816,ZOHAN,19161.0,59.783300,33.416700,1710.0,2005-12-28,5.2,-4.0,0.6,NaN,V2s5SVFVNHhPVEUyTVE9PQ==
4085817,ZOHAN,19161.0,59.783300,33.416700,1710.0,2005-12-29,-1.0,-6.2,-3.6,NaN,V2s5SVFVNHhPVEUyTVE9PQ==
4085818,ZOHAN,19161.0,59.783300,33.416700,1710.0,2005-12-30,-2.0,-7.0,-4.5,NaN,V2s5SVFVNHhPVEUyTVE9PQ==


<h1>
    <center>
        <font color='#F4EEE0'>
            Stations Information
        </font>
    </center>
</h1>

In [17]:
gb = ["name", "id", "lat", "lon", "alt", "code"]

station_info = data.groupby(by = gb)\
    .agg(
        date_min = ('date', 'min'),
        date_max = ('date', 'max'),
        n_days = ('date', 'size'),
        tmax_missing = ('tmax', lambda x: round((sum(x.isna()) / len(x)) * 100, 1)),
        tmin_missing = ('tmin', lambda x: round((sum(x.isna()) / len(x)) * 100, 1)),
        tm_missing = ('tm', lambda x: round((sum(x.isna()) / len(x)) * 100, 1)),
        precip_missing = ('precip', lambda x: round((sum(x.isna()) / len(x)) * 100, 1)),
    )\
    .sort_values(by = ["name", "id"])\
    .reset_index()

station_info

,name,id,lat,lon,alt,code,date_min,date_max,n_days,tmax_missing,tmin_missing,tm_missing,precip_missing
0,ABADAN,40831.0,30.377222,48.214722,6.6,UVVKQlJFRk9OREE0TXpFPQ==,2000-01-01,2022-07-12,8229,0.6,0.9,0.0,0.6
1,ABADCHI-FARIDAN,19264.0,32.716700,50.683100,2100.0,UVVKQlJFTklTUzFHUVZKSlJFRk9NVGt5TmpRPQ==,2014-10-14,2015-07-12,2,50.0,50.0,50.0,100.0
2,ABADE TASHK,19689.0,29.800000,53.720000,1601.0,UVVKQlJFVWdWRUZUU0VzeE9UWTRPUT09,2000-06-01,2022-07-12,7600,0.3,0.3,0.3,100.0
3,ABADEH,40818.0,31.198333,52.616389,2030.0,UVVKQlJFVklOREE0TVRnPQ==,2000-01-01,2022-07-12,8229,0.6,0.7,0.0,0.3
4,ABALI,40755.0,35.750000,51.883333,2465.0,UVVKQlRFazBNRGMxTlE9PQ==,2000-01-01,2022-07-12,8229,1.2,1.5,0.0,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,ZAVAREH,19156.0,33.440000,52.470000,995.0,V2tGV1FWSkZTREU1TVRVMg==,2019-11-04,2022-07-12,968,0.4,19.6,19.6,72.2
875,ZAVE,18839.0,35.480000,59.870000,1383.0,V2tGV1JURTRPRE01,2020-10-04,2022-07-12,466,4.5,4.5,4.5,100.0
876,ZIARATALI,19912.0,27.740000,57.230000,460.0,V2tsQlVrRlVRVXhKTVRrNU1UST0=,2022-06-23,2022-07-12,20,100.0,100.0,100.0,100.0
877,ZIDASHTE TALEGHAN,18554.0,36.130000,50.680000,2255.0,V2tsRVFWTklWRVVnVkVGTVJVZElRVTR4T0RVMU5BPT0=,2019-02-02,2022-07-12,1180,2.0,3.6,0.4,68.4


In [18]:
data.drop(columns=["name", "id", "lat", "lon", "alt"], inplace=True)
data.to_pickle("data.pkl")

station_info.to_pickle("stations.pkl")